In [1]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [2]:
ee.Initialize()

daymet = ee.ImageCollection('NASA/ORNL/DAYMET_V4') \
     .filter(ee.Filter.date('2000-01-01', '2000-12-31')) \
     .mean() ##not sure if this is the best way to do things!

dm_precip = daymet.select('prcp')
dm_maxtemp = daymet.select('tmax')
dm_mintemp = daymet.select('tmin')
dm_daylight = daymet.select('dayl')
dm_par = daymet.select('srad')
dm_snow = daymet.select('swe')
dm_vapor = daymet.select('vp')

prcpVis = {
  'min': -40.0,
  'max': 30.0,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = geemap.Map(center=[40,-100], zoom=4)
Map.addLayer(dm_precip, prcpVis, "Precipitation")
Map.addLayer(dm_maxtemp, prcpVis, "Maxtemp")
Map.addLayer(dm_mintemp, prcpVis, "Mintemp")
Map.addLayer(dm_daylight, prcpVis, "Daylight")
Map.addLayer(dm_par, prcpVis, "PAR")
Map.addLayer(dm_snow, prcpVis, "Snowpack water?")
Map.addLayer(dm_vapor, prcpVis, "Vaporpressure")
Map

#     .filter(ee.Filter.calendarRange(9, 11,'month')) \
#     .filterBounds(ga_pts) \
#     .filterMetadata('CLOUD_COVER', 'less_than', 100) \
#     .sort('CLOUD_COVER', True)

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
##Exporting pixel values from points
coords_year = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Daymet/GA/Coords_year/NoFlower/coords_2000.csv'
##NoFlower folder is identical to Coords_year folder, with flowering status removed (confuses csv_to_ee function)
fc_coords = geemap.csv_to_ee(coords_year, latitude = "Latitude", longitude = "Longitude")

Map.addLayer(fc_coords, {}, "fc_coords")

##export data
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'daymet2000.csv')
#out_shp = os.path.join(out_dir, 'points.shp') ##shapefile exporting

geemap.extract_values_to_points(fc_coords, dm_precip, out_csv) ##only works for single images - maybe for loop?


EEException: Image.reduceRegions: The default WGS84 projection is invalid for aggregations. Specify a scale or crs & crs_transform.

In [5]:
##From website: https://towardsdatascience.com/read-netcdf-data-with-python-901f7ff61648
##Also try: http://unidata.github.io/netcdf4-python/

## Also see: https://stackoverflow.com/questions/44359869/convert-netcdf-file-to-csv-or-text-using-python
import netCDF4 as nc4
import pandas as pd
fn = 'E:/Daymet/Daymet_V4_Stn_Level_CrossVal_1850/data/daymet_v4_stnxval_prcp_hi_2000.nc'
nc = nc4.Dataset(fn, mode='r')
print(nc.variables.keys())

lat = nc.variables['stn_lat'][:]
lon = nc.variables['stn_lon'][:]

time_var = nc.variables['time']
dtime = nc4.num2date(time_var[:],time_var.units)
precip = nc.variables['obs'][:] ##'obs' was originally 'precip' in example - make sure this is what you want

display(precip)
##a pandas.Series designed for time series of a 2D lat,lon grid
# precip_ts = pd.Series(precip, index=dtime) 
# display(precip_ts)

# precip_ts.to_csv('precip.csv',index=True, header=True)


dict_keys(['pred', 'obs', 'stns', 'station_id', 'station_name', 'stnx', 'stny', 'stnz', 'time', 'time_bnds', 'stn_lon', 'stn_lat'])


masked_array(
  data=[[--, --, --, ..., --, --, --],
        [0.3, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, ..., --, --, --],
        ...,
        [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, ..., 0.0, 0.5, 0.0],
        [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0]],
  mask=[[ True,  True,  True, ...,  True,  True,  True],
        [False, False, False, ..., False, False, False],
        [False, False, False, ...,  True,  True,  True],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],
  fill_value=-9999.0)